In [ ]:
pip install streamlit pdfplumber scikit-learn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 49.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 49.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 31.0 MB/s eta 0:00:00
  Attempting uninstall: cachetools
    Found existing installation: cachetools 7.0.0
    Uninstalling cachetools-7.0.0:
      Successfully uninstalled cachetools-7.0.0


In [ ]:
import os
import zipfile
import pdfplumber
import re
import numpy as np

from google.colab import files
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [ ]:
uploaded = files.upload()


Saving resume.zip to resume (3).zip


In [ ]:
zip_filename = list(uploaded.keys())[0]

with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
    zip_ref.extractall("/content/resumes")

print("ZIP Extracted Successfully")


ZIP Extracted Successfully


In [ ]:
base_path = "/content/resumes"

# Detect inner folder automatically
subfolders = os.listdir(base_path)

if len(subfolders) == 1 and os.path.isdir(os.path.join(base_path, subfolders[0])):
    folder_path = os.path.join(base_path, subfolders[0])
else:
    folder_path = base_path

print("Using folder path:", folder_path)


Using folder path: /content/resumes/resume


In [ ]:
def extract_text_from_pdf(file_path):
    text = ""
    with pdfplumber.open(file_path) as pdf:
        for page in pdf.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text + " "
    return text


In [ ]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z0-9\s]', ' ', text)
    return text


In [ ]:
def skill_match_score(resume_text, skills):
    matched = []
    for skill in skills:
        if skill in resume_text:
            matched.append(skill)
    score = len(matched) / len(skills)
    return score, matched


In [ ]:
job_description = """
We are looking for a Data Scientist with strong experience in Python,
Machine Learning, Deep Learning, Natural Language Processing (NLP),
SQL, Data Analysis, Pandas, NumPy, Scikit-learn,
data preprocessing, model building, and model deployment.
"""

cleaned_jd = clean_text(job_description)


In [ ]:
required_skills = [
    "python",
    "machine learning",
    "deep learning",
    "nlp",
    "sql",
    "pandas",
    "numpy",
    "scikit learn",
    "data analysis",
    "model deployment"
]


In [ ]:
resume_texts = []
candidate_names = []

for file in os.listdir(folder_path):
    if file.endswith(".pdf"):
        file_path = os.path.join(folder_path, file)
        raw_text = extract_text_from_pdf(file_path)
        cleaned = clean_text(raw_text)

        resume_texts.append(cleaned)
        candidate_names.append(file.replace(".pdf", ""))

print("Total resumes loaded:", len(resume_texts))


Total resumes loaded: 3


In [ ]:
documents = resume_texts + [cleaned_jd]

vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(documents)

jd_vector = tfidf_matrix[-1]
resume_vectors = tfidf_matrix[:-1]

tfidf_scores = cosine_similarity(jd_vector, resume_vectors)[0]


In [ ]:
final_results = []

for i in range(len(resume_texts)):

    skill_score, matched_skills = skill_match_score(
        resume_texts[i], required_skills
    )

    tfidf_score = tfidf_scores[i]

    final_score = (0.6 * tfidf_score) + (0.4 * skill_score)

    final_results.append(
        (
            candidate_names[i],
            tfidf_score,
            skill_score,
            final_score,
            matched_skills
        )
    )


In [ ]:
final_results = sorted(final_results, key=lambda x: x[3], reverse=True)

print("\n🚀 DATA SCIENCE ATS RANKING:\n")

for rank, (name, tfidf, skill, final, matched) in enumerate(final_results, 1):

    print(f"Rank {rank}: {name}")
    print(f"  TF-IDF Similarity: {tfidf*100:.2f}%")
    print(f"  Skill Match: {skill*100:.2f}%")
    print(f"  Final ATS Score: {final*100:.2f}%")
    print(f"  Matched Skills: {matched}")
    print("-"*60)



🚀 DATA SCIENCE ATS RANKING:

Rank 1: Sushil-Suresh-Mandhare-FlowCV-Resume-20251026
  TF-IDF Similarity: 36.33%
  Skill Match: 70.00%
  Final ATS Score: 49.80%
  Matched Skills: ['python', 'machine learning', 'deep learning', 'sql', 'pandas', 'numpy', 'data analysis']
------------------------------------------------------------
Rank 2: NewAkshay1
  TF-IDF Similarity: 13.87%
  Skill Match: 20.00%
  Final ATS Score: 16.32%
  Matched Skills: ['python', 'sql']
------------------------------------------------------------
Rank 3: avi resume
  TF-IDF Similarity: 7.14%
  Skill Match: 10.00%
  Final ATS Score: 8.28%
  Matched Skills: ['sql']
------------------------------------------------------------
